## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,34.81,68,2,17.60,clear sky
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,75.06,73,75,3.44,light intensity drizzle
2,2,Rikitea,PF,-23.1203,-134.9692,72.59,80,100,27.94,light rain
3,3,Nizwa,OM,22.9333,57.5333,96.55,29,83,6.31,broken clouds
4,4,San Quintin,MX,30.4833,-115.9500,77.31,69,56,8.48,broken clouds


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the minimum temperature you would like for your vacation "))
max_temp = float(input("Please enter the maximum temperature you would like for your vacation "))

Please enter the minimum temperature you would like for your vacation 75
Please enter the maximum temperature you would like for your vacation 90


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,75.06,73,75,3.44,light intensity drizzle
4,4,San Quintin,MX,30.4833,-115.9500,77.31,69,56,8.48,broken clouds
7,7,Yenagoa,NG,4.9247,6.2642,76.30,86,100,6.58,light rain
8,8,Tura,IN,25.5198,90.2201,75.45,96,28,6.33,scattered clouds
9,9,Paciran,ID,-6.8767,112.3761,77.68,83,56,7.23,broken clouds
11,11,Butaritari,KI,3.0707,172.7902,80.46,80,24,13.22,few clouds
21,21,Coahuayana,MX,18.7333,-103.6833,85.77,70,71,6.15,broken clouds
23,23,Samarai,PG,-10.6167,150.6667,79.25,72,99,14.74,overcast clouds
24,24,Minden,DE,52.2833,8.9167,75.69,44,20,8.05,few clouds
28,28,Puerto Ayora,EC,-0.7393,-90.3518,76.96,82,84,8.99,broken clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                261
City                   261
Country                261
Lat                    261
Lng                    261
Max Temp               261
Humidity               261
Cloudiness             261
Wind Speed             261
Current Description    261
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,75.06,73,75,3.44,light intensity drizzle
4,4,San Quintin,MX,30.4833,-115.9500,77.31,69,56,8.48,broken clouds
7,7,Yenagoa,NG,4.9247,6.2642,76.30,86,100,6.58,light rain
8,8,Tura,IN,25.5198,90.2201,75.45,96,28,6.33,scattered clouds
9,9,Paciran,ID,-6.8767,112.3761,77.68,83,56,7.23,broken clouds
...,...,...,...,...,...,...,...,...,...,...
689,689,Surt,LY,31.2089,16.5887,82.80,71,0,11.86,clear sky
697,697,Zhumadian,CN,32.9794,114.0294,84.79,74,97,6.17,overcast clouds
698,698,Qabaqcol,AZ,41.7200,46.2647,76.01,60,63,7.25,broken clouds
705,705,Nicoya,CR,10.1483,-85.4520,78.55,84,99,5.61,overcast clouds


In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.06,light intensity drizzle,-22.9661,-42.0278,
4,San Quintin,MX,77.31,broken clouds,30.4833,-115.9500,
7,Yenagoa,NG,76.30,light rain,4.9247,6.2642,
8,Tura,IN,75.45,scattered clouds,25.5198,90.2201,
9,Paciran,ID,77.68,broken clouds,-6.8767,112.3761,
11,Butaritari,KI,80.46,few clouds,3.0707,172.7902,
21,Coahuayana,MX,85.77,broken clouds,18.7333,-103.6833,
23,Samarai,PG,79.25,overcast clouds,-10.6167,150.6667,
24,Minden,DE,75.69,few clouds,52.2833,8.9167,
28,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row  in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.06,light intensity drizzle,-22.9661,-42.0278,Pousada Porto Praia
4,San Quintin,MX,77.31,broken clouds,30.4833,-115.9500,Don Eddie's / Hotel - Restaurant
7,Yenagoa,NG,76.30,light rain,4.9247,6.2642,Aridolf Resort Wellness and Spa
8,Tura,IN,75.45,scattered clouds,25.5198,90.2201,Hotel RIKMAN Continental
9,Paciran,ID,77.68,broken clouds,-6.8767,112.3761,Tanjung Kodok Beach Resort


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F </dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))